In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.optim as optim
torch.manual_seed(1)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ai-w6-p1/sample_submission.csv
/kaggle/input/2021-ai-w6-p1/pca_test.csv
/kaggle/input/2021-ai-w6-p1/pca_train.csv
/kaggle/input/2021-ai-w6-p1/train_label.csv


In [2]:
X = pd.read_csv("/kaggle/input/2021-ai-w6-p1/pca_train.csv")
test = pd.read_csv("/kaggle/input/2021-ai-w6-p1/pca_test.csv")
y = pd.read_csv("/kaggle/input/2021-ai-w6-p1/train_label.csv").drop(["id"], axis = 1)

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
X = torch.FloatTensor(X.to_numpy()).to(device)
test = torch.FloatTensor(test.to_numpy()).to(device)
y = torch.LongTensor(y.to_numpy()).squeeze(1).to(device)

In [5]:
## 모델 설계
linear1 = torch.nn.Linear(256, 10, bias=True).to(device)
linear2 = torch.nn.Linear(10, 10, bias=True).to(device)
linear3 = torch.nn.Linear(10, 198, bias=True).to(device)

In [6]:
model = torch.nn.Sequential(linear1, linear2, linear3)

In [7]:
nb_class = 198
nb_data = len(y)

loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

for stop in range(20000):
    # 그래디언트 초기화
    optimizer.zero_grad()
    hypothesis = model(X)
    cost = loss(hypothesis, y)
    
    # Backward 계산 
    cost.backward()
    # 가중치 갱신
    optimizer.step()

    if stop % 1000 == 0:
        print(stop, cost.item())


0 5.406797885894775
1000 2.796003818511963
2000 2.358250856399536
3000 2.140415906906128
4000 1.9798815250396729
5000 1.870087742805481
6000 1.7880890369415283
7000 1.722005009651184
8000 1.6666585206985474
9000 1.6201112270355225
10000 1.5806831121444702
11000 1.547636866569519
12000 1.5201767683029175
13000 1.4971165657043457
14000 1.4772915840148926
15000 1.459898591041565
16000 1.4446022510528564
17000 1.4311703443527222
18000 1.419219970703125
19000 1.4084396362304688


In [8]:
hypothesis = torch.nn.functional.softmax(model(X))
predict = torch.argmax(hypothesis, dim=1) # argmax : 가장 큰 값의 인덱스 리턴

correct_prediction = predict.float() == y
print(correct_prediction)
accuracy = correct_prediction.sum().item()/len(correct_prediction)
print('{:2.2f}%'.format(accuracy*100))

tensor([False, False,  True,  ...,  True,  True,  True], device='cuda:0')
66.48%


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


In [9]:
hypothesis =  torch.nn.functional.softmax(model(test))
predict = torch.argmax(hypothesis, dim=1) 


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


In [10]:
predict

tensor([113,  91,  68,  ...,  23,  68, 117], device='cuda:0')

In [11]:
sample = pd.read_csv("/kaggle/input/2021-ai-w6-p1/sample_submission.csv")

In [12]:
test.shape

torch.Size([13629, 256])

In [13]:
sample['label'] = predict.cpu().detach().numpy()

In [14]:
sample.head()

,id,label
0,0,113
1,1,91
2,2,68
3,3,28
4,4,121


In [15]:
sample.to_csv("submit.csv", index = False)